In [1]:
pip install plotly kaleido pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 106 kB/s 
     |████████████████████████████████| 14.2 MB 36.8 MB/s 


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
from pulp import *

In [4]:
predicted_fwds = pd.read_csv("forwards_points.csv",index_col=0)
predicted_fwds["name"] = predicted_fwds["name"].replace({
        "Diogo Teixeira da Silva": "Diogo jota",
        "Cristiano Ronaldo dos Santos Aveiro": "Cristaino Ronaldo",
        "Gabriel Fernando de Jesus": "Gabriel Jesus",
        "Darwin Núñez Ribeiro": "Darwin Núñez",})
injured=["Richarlison de Andrade","Diogo jota"]
predicted_fwds=predicted_fwds[~predicted_fwds["name"].isin(injured)]
predicted_fwds["position"]="FWD"
max_fwds=predicted_fwds["points"].max()
predicted_fwds["points"]=(predicted_fwds["points"])/max_fwds

In [5]:
predicted_defs = pd.read_csv("defenders_points.csv",index_col=0)
predicted_defs["position"]="DEF"
predicted_defs=predicted_defs[~predicted_defs["name"].isin(injured)]
max_defs=predicted_defs["points"].max()
predicted_defs["points"]=(predicted_defs["points"])/max_defs

In [6]:
predicted_mids = pd.read_csv("midfielders_points.csv",index_col=0)
predicted_mids["position"]="MID"
max_mids=predicted_mids["points"].max()
predicted_mids["points"]=(predicted_mids["points"])/max_mids

In [7]:
predicted_gks = pd.read_csv("goalkeepers_points.csv",index_col=0)
predicted_gks["position"]="GK"
max_gks=predicted_gks["points"].max()
predicted_gks["points"]=(predicted_gks["points"])/max_gks

In [8]:
predictions=pd.concat([predicted_fwds,predicted_defs,predicted_mids,predicted_gks])

<h1><b>Fantasy Premier league</b></h1>

In [9]:
POS = predictions["position"].unique()
CLUBS = predictions["team_x"].unique()
BUDGET = 990
pos_available = {
    'DEF': 5,
    'FWD': 3,
    'MID': 5,
    'GK': 2,
}

In [10]:
# Initialize Variables
print(predictions.index)
names = [predictions["name"].loc[i] for i in predictions.index]
print(1)
teams = [predictions["team_x"].loc[i] for i in predictions.index]
positions = [predictions["position"].loc[i] for i in predictions.index]
prices = [predictions["value"].loc[i] for i in predictions.index]
points = [predictions["points"].loc[i] for i in predictions.index]
players = [LpVariable("player_" + str(i), cat="Binary") for i in predictions["name"]]
#print(predictions)

Index(['Harry Kane2022-08-06 14:00:00+00:00',
       'Cristiano Ronaldo dos Santos Aveiro2022-08-07 13:00:00+00:00',
       'Ollie Watkins2022-08-06 14:00:00+00:00',
       'Jamie Vardy2022-08-07 13:00:00+00:00',
       'Danny Ings2022-08-06 14:00:00+00:00',
       'Kai Havertz2022-08-06 16:30:00+00:00',
       'Ivan Toney2022-08-07 13:00:00+00:00',
       'Chris Wood2022-08-06 14:00:00+00:00',
       'Gabriel Fernando de Jesus2022-08-05 19:00:00+00:00',
       'Callum Wilson2022-08-06 14:00:00+00:00',
       ...
       'Martin Dubravka2022-08-06 14:00:00+00:00',
       'Aaron Ramsdale2022-08-05 19:00:00+00:00',
       'Jordan Pickford2022-08-06 16:30:00+00:00',
       'Robert Sánchez2022-08-07 13:00:00+00:00',
       'Ederson Santana de Moraes2022-08-07 15:30:00+00:00',
       'David Raya Martin2022-08-07 13:00:00+00:00',
       'Edouard Mendy2022-08-06 16:30:00+00:00',
       'Vicente Guaita2022-08-05 19:00:00+00:00',
       'Lukasz Fabianski2022-08-07 15:30:00+00:00',
       'Sam Jo

In [11]:
# Initialize the problem
prob = LpProblem("FPL Player Choices", LpMaximize)

# Define the objective
prob += lpSum(players[i] * points[i] for i in range(len(predictions)))
# Objective

# Build the constraints
prob += lpSum(players[i] * predictions.value[predictions.index[i]] for i in range(len(predictions))) <= BUDGET # Budget Limit

for pos in POS:
 prob += lpSum(players[i] for i in range(len(predictions)) if positions[i] == pos) == pos_available[pos] # Position Limit

for club in CLUBS:
 prob += lpSum(players[i] for i in range(len(predictions)) if teams[i] == club) <= 2# Club Limit

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [12]:
prob.solve()

1

In [13]:
for v in prob.variables():
    if v.varValue != 0:
        print(v)

player_Aaron_Wan_Bissaka
player_Ben_Chilwell
player_Cristaino_Ronaldo
player_Declan_Rice
player_Harry_Arter
player_Harry_Kane
player_Illan_Meslier
player_James_Ward_Prowse
player_Jefferson_Lerma_Solís
player_Joel_Matip
player_Matt_Doherty
player_Mohamed_Salah
player_Nick_Pope
player_Ollie_Watkins
player_Timothy_Castagne


In [14]:
score = str(prob.objective)
constraint = [str(const) for const in prob.constraints.values()][0]
for v in prob.variables():
 score = score.replace(v.name, str(v.varValue))
 constraint = constraint.replace(v.name, str(v.varValue))

score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

print("Constraint: ")
print(constraint_pretty + " = " + str(eval(constraint_pretty)))
print()
print("Score: ")
print(score_pretty + "= " + str(eval(score_pretty)))

Constraint: 
45*1.0 + 60*1.0 + 105*1.0 + 50*1.0 + 45*1.0 + 115*1.0 + 45*1.0 + 65*1.0 + 50*1.0 + 60*1.0 + 50*1.0 + 130*1.0 + 50*1.0 + 75*1.0 + 45*1.0 = 990.0

Score: 
0.5625558892201893*1.0 + 0.6360353340482621*1.0 + 0.8009023565862402*1.0 + 0.1989603732498932*1.0 + 0.1740486898674216*1.0 + 0.2754376814879696*1.0 + 0.20804814090354692*1.0 + 0.6170590447422813*1.0 + 0.959909588608369*1.0 + 0.687488325728131*1.0 + 0.5560572867296779*1.0= 5.676502711171982


<h1><b>PLOT TOP PLAYERS</b></h1>

In [15]:
no_plotted_players=10

FWDS

In [16]:
fig = px.bar(predicted_fwds.head(no_plotted_players).sort_values("points", ascending=True), x="name", y="points",color="name" ,text="name", title="Long-Form Input")
fig.update_xaxes(visible=False, showticklabels=False)
fig.show()
fig.write_image("plots/forwards.png")

MIDS

In [17]:
fig = px.bar(predicted_mids.head(no_plotted_players).sort_values("points", ascending=True), x="name", y="points",color="name" ,text="name", title="Long-Form Input")
fig.update_xaxes(visible=False, showticklabels=False)
fig.show()
fig.write_image("plots/midfielders.png")

defs

In [18]:
fig = px.bar(predicted_defs.head(no_plotted_players).sort_values("points", ascending=True), x="name", y="points",color="name" ,text="name", title="Long-Form Input")
fig.update_xaxes(visible=False, showticklabels=False)
fig.show()
fig.write_image("plots/defenders.png")

GKS

In [20]:
fig = px.bar(predicted_gks.head(no_plotted_players).sort_values("points", ascending=True), x="name", y="points",color="name" ,text="name", title="Long-Form Input")
fig.update_xaxes(visible=False, showticklabels=False)
fig.show()
fig.write_image("plots/goalkeepers.png")